In [1]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout, Conv2D, Flatten, Dense, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping 
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.optimizers import RMSprop

In [2]:
# Load your custom dataset and labels
# Set the path to the root dataset directory
root_dir = 'dataset/'

# Initialize lists to store images and labels
images = []
labels = []

# Recursively traverse the directory tree
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Loop through the files in the current directory
    for file in filenames:
        if file.endswith('.png'):  # assuming all images are in png format
            image = cv2.imread(os.path.join(dirpath, file))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # convert to grayscale
            image = cv2.resize(image, (24, 24))  # resize to match your model's input shape
            images.append(image)

            # Extract the label from the file name
            if file[16] == '1':
                label = 1
            else:
                label = 0
            labels.append(label)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Reshape the images for the Conv2D layer
images = images.reshape(images.shape[0], 24, 24, 1)

# Normalize the pixel values between 0 and 1
images = images / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [3]:

# %%
# Convert the labels to categorical format (one-hot encoding)
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

# Create the model with reduced regularization and simpler architecture
model = Sequential([
    Conv2D(8, kernel_size=(3, 3), activation='sigmoid', input_shape=(24, 24, 1)),
    MaxPooling2D(pool_size=(1, 1)),
    Dropout(0.5),
    Flatten(),
    Dense(32, activation='sigmoid'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

optimizer = Adam(learning_rate=0.001)  # You can experiment with different learning rates
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

# Model training with Early Stopping
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])


Epoch 1/10
2123/2123 [==============================] - 33s 15ms/step - loss: 0.6517 - accuracy: 0.5743 - val_loss: 0.3891 - val_accuracy: 0.8390
Epoch 2/10
2123/2123 [==============================] - 33s 15ms/step - loss: 0.3654 - accuracy: 0.8518 - val_loss: 0.2649 - val_accuracy: 0.8969
Epoch 3/10
2123/2123 [==============================] - 33s 16ms/step - loss: 0.3016 - accuracy: 0.8849 - val_loss: 0.2433 - val_accuracy: 0.9074
Epoch 4/10
2123/2123 [==============================] - 31s 14ms/step - loss: 0.2728 - accuracy: 0.8958 - val_loss: 0.2060 - val_accuracy: 0.9216
Epoch 5/10
2123/2123 [==============================] - 31s 15ms/step - loss: 0.2543 - accuracy: 0.9038 - val_loss: 0.1948 - val_accuracy: 0.9254
Epoch 6/10
2123/2123 [==============================] - 30s 14ms/step - loss: 0.2430 - accuracy: 0.9086 - val_loss: 0.1822 - val_accuracy: 0.9329
Epoch 7/10
2123/2123 [==============================] - 30s 14ms/step - loss: 0.2355 - accuracy: 0.9122 - val_loss: 0.1865 -

In [4]:
# %%
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Save the model
model.save('models/cnnCat2_custom_low_accuracy.h5', overwrite=True)

  1/531 [..............................] - ETA: 33s - loss: 0.1422 - accuracy: 0.9062

531/531 [==============================] - 3s 5ms/step - loss: 0.1578 - accuracy: 0.9408
Test loss: 0.15777979791164398
Test accuracy: 0.9408127069473267
